In [1]:
cd ..

/home/abhishekabhishek/git/UnsupGenModbyMPS


In [2]:
import numpy as np
from MPScumulant import MPS_c

### Trying to do the following 
1. load the trained MPS
2. check its bond dimensions
3. try to zero-pad it
4. try to left-canonicalize again

In [3]:
m = MPS_c(16)
m.loadMPS('BS-MPS')

In [4]:
len(m.matrices)

16

In [5]:
# check the properties of the matrices in the MPS
for i in range(len(m.matrices)):
    tn_core = np.swapaxes(m.matrices[i], 0, 1)
    print(i, tn_core.shape)

0 (2, 1, 2)
1 (2, 2, 4)
2 (2, 4, 8)
3 (2, 8, 15)
4 (2, 15, 16)
5 (2, 16, 16)
6 (2, 16, 16)
7 (2, 16, 15)
8 (2, 15, 16)
9 (2, 16, 16)
10 (2, 16, 16)
11 (2, 16, 15)
12 (2, 15, 8)
13 (2, 8, 4)
14 (2, 4, 2)
15 (2, 2, 1)


In [6]:
m.bond_dimension

array([ 2,  4,  8, 15, 16, 16, 16, 15, 16, 16, 16, 15,  8,  4,  2,  1],
      dtype=int16)

In [7]:
len(m.bond_dimension)

16

Try to left-canonicalize the trained MPS - maybe let's not do this before padding

In [8]:
# m.left_cano()

In [9]:
# check the properties of the matrices in the MPS
for i in range(len(m.matrices)):
    #tn_core = np.swapaxes(m.matrices[i], 0, 1)
    tn_core = m.matrices[i]
    print(i, tn_core.shape)

0 (1, 2, 2)
1 (2, 2, 4)
2 (4, 2, 8)
3 (8, 2, 15)
4 (15, 2, 16)
5 (16, 2, 16)
6 (16, 2, 16)
7 (16, 2, 15)
8 (15, 2, 16)
9 (16, 2, 16)
10 (16, 2, 16)
11 (16, 2, 15)
12 (15, 2, 8)
13 (8, 2, 4)
14 (4, 2, 2)
15 (2, 2, 1)


It looks like nothing changed after performing the left canonicalization of the MPS in terms of the tensor shapes - try to zero pad and do it again

this is where the trouble starts - we need to figure out how to zero-pad and left-cano afterwards

Idea - Try to do it in their code and see if it works - otherwise do the zero padding and initialize in the TensorNetwork library and do the canonicalization there

**Also do note that all core tensors have atleast one bond dim which is a power of two which kind of takes care of the (odd, odd) special case**

In [12]:
# try to zero pad the TN core to make them powers of two
new_core_mats = m.matrices.copy()
new_bond_dims = m.bond_dimension.copy()
for i in range(1, len(m.matrices)-1):
    core_mat = m.matrices[i]
    pad_width_list = []
    for dim in range(len(core_mat.shape)):
        if core_mat.shape[dim]%2 == 0:
            pad_width_list.append((0, 0))
        else:
            pad_width_list.append((0, 1))
    core_mat = np.pad(core_mat, pad_width_list, mode='constant',
                           constant_values=0)
    new_core_mats[i] = core_mat
    new_bond_dims[i-1] = core_mat.shape[0]
    print(f"i = {i}, {core_mat.shape}, {core_mat.shape[0]}")          

i = 1, (2, 2, 4), 2
i = 2, (4, 2, 8), 4
i = 3, (8, 2, 16), 8
i = 4, (16, 2, 16), 16
i = 5, (16, 2, 16), 16
i = 6, (16, 2, 16), 16
i = 7, (16, 2, 16), 16
i = 8, (16, 2, 16), 16
i = 9, (16, 2, 16), 16
i = 10, (16, 2, 16), 16
i = 11, (16, 2, 16), 16
i = 12, (16, 2, 8), 16
i = 13, (8, 2, 4), 8
i = 14, (4, 2, 2), 4


In [15]:
# create a new MPS with padded core tensors
m_pad = MPS_c(16)
m_pad.matrices = new_core_mats
m_pad.bond_dimension = new_bond_dims

In [16]:
# check the properties of the matrices in the MPS
for i in range(len(m_pad.matrices)):
    #tn_core = np.swapaxes(m.matrices[i], 0, 1)
    tn_core = m_pad.matrices[i]
    print(i, tn_core.shape)
    
m_pad.bond_dimension

0 (1, 2, 2)
1 (2, 2, 4)
2 (4, 2, 8)
3 (8, 2, 16)
4 (16, 2, 16)
5 (16, 2, 16)
6 (16, 2, 16)
7 (16, 2, 16)
8 (16, 2, 16)
9 (16, 2, 16)
10 (16, 2, 16)
11 (16, 2, 16)
12 (16, 2, 8)
13 (8, 2, 4)
14 (4, 2, 2)
15 (2, 2, 1)


array([ 2,  4,  8, 16, 16, 16, 16, 16, 16, 16, 16, 16,  8,  4,  2,  1],
      dtype=int16)

In [17]:
# compare the above with the original MPS
# check the properties of the matrices in the MPS
for i in range(len(m.matrices)):
    #tn_core = np.swapaxes(m.matrices[i], 0, 1)
    tn_core = m.matrices[i]
    print(i, tn_core.shape)
    
m.bond_dimension

0 (1, 2, 2)
1 (2, 2, 4)
2 (4, 2, 8)
3 (8, 2, 15)
4 (15, 2, 16)
5 (16, 2, 16)
6 (16, 2, 16)
7 (16, 2, 15)
8 (15, 2, 16)
9 (16, 2, 16)
10 (16, 2, 16)
11 (16, 2, 15)
12 (15, 2, 8)
13 (8, 2, 4)
14 (4, 2, 2)
15 (2, 2, 1)


array([ 2,  4,  8, 15, 16, 16, 16, 15, 16, 16, 16, 15,  8,  4,  2,  1],
      dtype=int16)

Try to left-canonicalize the padded MPS

In [18]:
m_pad.left_cano()

bond: 0
bond: 1
bond: 2
bond: 3
bond: 4
bond: 5
bond: 6
bond: 7
bond: 8
bond: 9
bond: 10
bond: 11
bond: 12
bond: 13
bond: 14


In [19]:
# check the properties of the matrices in the MPS (after left canonicalization)
for i in range(len(m_pad.matrices)):
    #tn_core = np.swapaxes(m.matrices[i], 0, 1)
    tn_core = m_pad.matrices[i]
    print(i, tn_core.shape)
    
m_pad.bond_dimension

0 (1, 2, 2)
1 (2, 2, 4)
2 (4, 2, 8)
3 (8, 2, 16)
4 (16, 2, 16)
5 (16, 2, 16)
6 (16, 2, 16)
7 (16, 2, 16)
8 (16, 2, 16)
9 (16, 2, 16)
10 (16, 2, 16)
11 (16, 2, 16)
12 (16, 2, 8)
13 (8, 2, 4)
14 (4, 2, 2)
15 (2, 2, 1)


array([ 2,  4,  8, 16, 16, 16, 16, 16, 16, 16, 16, 16,  8,  4,  2,  1],
      dtype=int16)

compare with the sizes and bdims before left canonicalization:

```
0 (1, 2, 2)
1 (2, 2, 4)
2 (4, 2, 8)
3 (8, 2, 16)
4 (16, 2, 16)
5 (16, 2, 16)
6 (16, 2, 16)
7 (16, 2, 16)
8 (16, 2, 16)
9 (16, 2, 16)
10 (16, 2, 16)
11 (16, 2, 16)
12 (16, 2, 8)
13 (8, 2, 4)
14 (4, 2, 2)
15 (2, 2, 1)
array([ 2,  4,  8, 16, 16, 16, 16, 16, 16, 16, 16, 16,  8,  4,  2,  1],
      dtype=int16)
```
#### It's the same - no $2^{n+1}$ stuff going as the decomposition paper claims - we can check if the resulting matrices are actually isometries